In [1]:
import numpy as np
import pandas_ta as ta
import pandas as pd
from datetime import datetime, timedelta, timezone
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
from Volatility import *
from  Volume import *

d:\Capstone_RL\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Import data

In [2]:
df_min = pd.read_csv('vn30f1m_3min.csv')
df_hour = pd.read_csv('vn30f1m_1hour.csv')
df_day = pd.read_csv('vn30f1m_1day.csv')

In [3]:
def data_backtest(data):
    new_data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    new_data.index = pd.to_datetime(data['Date'])
    new_data.index.name = None
    return new_data

In [4]:
def split_data(data):
    """
    Hàm này chia dữ liệu thành 2 phần: tập huấn luyện và tập hold out.

    Args:
    data (pandas.DataFrame): DataFrame chứa dữ liệu cần chia.

    Returns:
    pandas.DataFrame: DataFrame chứa dữ liệu tập huấn luyện.
    pandas.DataFrame: DataFrame chứa dữ liệu tập giữ lại.
    """
    # Chia dữ liệu thành 3 phần
    new_part = np.array_split(data, 3)

    # Access each part individually
    hold_out = new_part[2]
    train_data = pd.concat([new_part[0], new_part[1]], axis=0)

    return train_data, hold_out

In [5]:
def process_data(df):
    data = df.copy()
    data = data[~data.index.duplicated(keep='first')] # Handling duplicate
    
    data['Date'] = [str(i)[:10] for i in data.index]
    data['time'] = [str(i)[11:] for i in data.index]
    data_model = data.pivot(index = 'Date', columns = 'time', values = ['Open','High','Low','Close','Volume']).ffill(axis = 1).stack().reset_index() # Handling missing values

    return data_model

In [6]:
data_min = data_backtest(df_min)
data_hour = data_backtest(df_hour)
data_day = data_backtest(df_day)

In [7]:
train_min, valid_min = split_data(data_min)
train_hour, valid_hour = split_data(data_hour)
train_day, valid_day = split_data(data_day)

d:\Capstone_RL\.venv\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


### Mẫu

    def indicator(data):
        Hàm indicator
        formular

        return np.array(something)

    class CustomStrategy(Strategy):

        def init(self):
            Note that this returns each column as a row
            Then extends the length of each row on each iteration
            self.something = self.I(indicator, self.data)

        def next(self):

            sth1 = self.something[0]
            sth2 = self.something[2]

            if self.position:
                if something happens:
                    self.position.close()
            else:
                if something happens:
                    self.buy(size=0.7)
                
    bt = Backtest(data, CustomStrategy, cash=3000, commission=0.01, margin=0.13, hedging=True, exclusive_orders=False)
    stats = bt.run()
    stats

# Backtest

In [8]:
# from momen import *
# from other import *

### Volatility

##### Average True Range (ATR)


In [9]:
class ATR_Strategy(Strategy):
    period = 14

    def init(self):
        self.atr = self.I(average_true_range, self.data, self.period)

    def next(self):
        atr = self.atr[-1]
        close = self.data.Close[-1]
        upper = close + atr * 2  # Upper bound cho Take Profit
        lower = close - atr * 1.5  # Lower bound cho Stop Loss

        if self.position:
            if close < lower or close > upper:
                self.position.close()
        else:
            if atr > 0.7:
                self.buy(size=0.7)

bt = Backtest(train_day, ATR_Strategy, cash=3000, commission=0.01, hedging=True)
stats = bt.optimize(period=range(10, 30, 2), maximize='Equity Final [$]')
print(stats)


Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   98.205384
Equity Final [$]                     3688.094
Equity Peak [$]                      4246.694
Return [%]                          22.936467
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                     5.38016
Volatility (Ann.) [%]               16.636678
Sharpe Ratio                         0.323392
Sortino Ratio                        0.459746
Calmar Ratio                         0.201427
Max. Drawdown [%]                  -26.710234
Avg. Drawdown [%]                   -2.798979
Max. Drawdown Duration      795 days 00:00:00
Avg. Drawdown Duration       51 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      36.420463
Worst Trade [%]                     36.420463
Avg. Trade [%]                    

##### Bollinger Bands (BB)


In [11]:
class BB_Strategy(Strategy):
    window = 20
    multiplier = 2

    def init(self):
        self.upper_band, self.lower_band = self.I(bollinger_bands, self.data, self.window, self.multiplier)

    def next(self):
        close = self.data.Close[-1]
        upper_band = self.upper_band[-1]
        lower_band = self.lower_band[-1]

        if self.position:
            if close > upper_band or close < lower_band:
                self.position.close()
        else:
            if close < lower_band:
                self.buy(size=1)

bt = Backtest(train_day, BB_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True)
stats = bt.optimize(window=range(10, 30, 2), multiplier=[1.5, 2, 2.5, 3], maximize='Equity Final [$]')
print(stats)


Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   47.158524
Equity Final [$]                     3557.496
Equity Peak [$]                      3730.856
Return [%]                            18.5832
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                    4.420396
Volatility (Ann.) [%]                6.311895
Sharpe Ratio                         0.700328
Sortino Ratio                        0.997728
Calmar Ratio                         0.631002
Max. Drawdown [%]                   -7.005363
Avg. Drawdown [%]                   -0.908692
Max. Drawdown Duration      421 days 00:00:00
Avg. Drawdown Duration       26 days 00:00:00
# Trades                                    4
Win Rate [%]                             50.0
Best Trade [%]                     100.975525
Worst Trade [%]                    -13.857576
Avg. Trade [%]                    

##### Donchian Channel


In [13]:
class DonchianChannel_Strategy(Strategy):
    window = 20

    def init(self):
        self.upper_band, self.lower_band = self.I(donchian_channel, self.data, self.window)

    def next(self):
        close = self.data.Close[-1]
        upper_band = self.upper_band[-1]
        lower_band = self.lower_band[-1]

        if self.position:
            if close < lower_band or close > upper_band:
                self.position.close()
        else:
            if close > upper_band:
                self.buy(size=1)

bt = Backtest(train_day, DonchianChannel_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True)
stats = bt.optimize(window=range(10, 30, 2), maximize='Equity Final [$]')
print(stats)


Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                         0.0
Equity Final [$]                       3000.0
Equity Peak [$]                        3000.0
Return [%]                                0.0
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

d:\Capstone_RL\.venv\lib\site-packages\backtesting\backtesting.py:1384: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  best_params = heatmap.idxmax()


##### Keltner Channel (KC)


In [ ]:
class KeltnerChannel_Strategy(Strategy):
    window = 20
    multiplier = 2
    atr_window = 14

    def init(self):
        self.upper_band, self.lower_band = self.I(keltner_channel, self.data, self.window, self.multiplier, self.atr_window)

    def next(self):
        close = self.data.Close[-1]
        upper_band = self.upper_band[-1]
        lower_band = self.lower_band[-1]

        if self.position:
            if close > upper_band or close < lower_band:
                self.position.close()
        else:
            if close < lower_band:
                self.buy(size=1)

bt = Backtest(train_day, KeltnerChannel_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True)
stats = bt.optimize(window=range(10, 30, 2), multiplier=[1.5, 2, 2.5], atr_window=range(10, 20, 2), maximize='Equity Final [$]')
print(stats)


Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   33.200399
Equity Final [$]                      3577.78
Equity Peak [$]                      3838.519
Return [%]                          19.259333
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                    4.571169
Volatility (Ann.) [%]                5.512589
Sharpe Ratio                         0.829224
Sortino Ratio                        1.252509
Calmar Ratio                          0.65292
Max. Drawdown [%]                   -7.001112
Avg. Drawdown [%]                   -1.041185
Max. Drawdown Duration      421 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                   15
Win Rate [%]                        53.333333
Best Trade [%]                      47.751503
Worst Trade [%]                    -16.417776
Avg. Trade [%]                    

##### Ulcer Index (UI)


In [12]:
class UI_Strategy(Strategy):
    window = 14

    def init(self):
        self.ui = self.I(ulcer_index, self.data, self.window)

    def next(self):
        ui = self.ui[-1]
        close = self.data.Close[-1]
        upper = close * 1.05  # Take Profit
        lower = close * 0.95  # Stop Loss

        if self.position:
            if close < lower or close > upper:
                self.position.close()
        else:
            if ui < 5:
                self.buy(size=1)

bt = Backtest(train_day, UI_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True)
stats = bt.optimize(window=range(10, 30, 2), maximize='Equity Final [$]')
print(stats)


Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   90.628116
Equity Final [$]                     3419.999
Equity Peak [$]                      3699.299
Return [%]                          13.999967
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                    3.381083
Volatility (Ann.) [%]                   8.038
Sharpe Ratio                         0.420637
Sortino Ratio                         0.58185
Calmar Ratio                         0.285396
Max. Drawdown [%]                  -11.846988
Avg. Drawdown [%]                   -1.312869
Max. Drawdown Duration      421 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      48.347936
Worst Trade [%]                     48.347936
Avg. Trade [%]                    

### Volume

##### Accumulation/Distribution Index (ADI)


In [ ]:
class ADI_Strategy(Strategy):

    def init(self):
        self.adi = self.I(calc_acc_dist_index, self.data)

    def next(self):
        adi = self.adi[-1]
        close = self.data.Close[-1]
        upper = close * 1.05
        lower = close * 0.95

        if self.position:
            if close < lower or close > upper:
                self.position.close()
        else:
            if adi > 0:
                self.buy(size=1)

bt = Backtest(train_day, ADI_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True)
stats = bt.run()
print(stats)


Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   99.800598
Equity Final [$]                      3321.12
Equity Peak [$]                       3600.42
Return [%]                             10.704
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                    2.614234
Volatility (Ann.) [%]                8.452775
Sharpe Ratio                         0.309275
Sortino Ratio                         0.42391
Calmar Ratio                         0.191303
Max. Drawdown [%]                   -13.66543
Avg. Drawdown [%]                    -1.63086
Max. Drawdown Duration      795 days 00:00:00
Avg. Drawdown Duration       54 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      33.187953
Worst Trade [%]                     33.187953
Avg. Trade [%]                    

##### Chaikin Money Flow (CMF)

In [ ]:
class CMF_Strategy(Strategy):
    window = 20

    def init(self):
        self.cmf = self.I(calc_chaikin_money_flow, self.data, self.window)

    def next(self):
        cmf = self.cmf[-1]
        close = self.data.Close[-1]
        upper = close * 1.05
        lower = close * 0.95

        if self.position:
            if close < lower or close > upper:
                self.position.close()
        else:
            if cmf > 0:
                self.buy(size=1)

bt = Backtest(train_day, CMF_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True)
stats = bt.optimize(window=range(10, 30, 2), maximize='Equity Final [$]')
print(stats)


Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   92.123629
Equity Final [$]                     3367.075
Equity Peak [$]                      3646.375
Return [%]                          12.235833
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                    2.972724
Volatility (Ann.) [%]                8.155873
Sharpe Ratio                         0.364489
Sortino Ratio                         0.50177
Calmar Ratio                         0.246607
Max. Drawdown [%]                  -12.054513
Avg. Drawdown [%]                   -1.434012
Max. Drawdown Duration      421 days 00:00:00
Avg. Drawdown Duration       40 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      39.829106
Worst Trade [%]                     39.829106
Avg. Trade [%]                    

##### Ease of Movement (EoM)


In [ ]:
class EoM_Strategy(Strategy):
    window = 14

    def init(self):
        self.eom = self.I(calc_ease_of_movement, self.data, self.window)

    def next(self):
        eom = self.eom[-1]
        close = self.data.Close[-1]
        upper = close * 1.05
        lower = close * 0.95

        if self.position:
            if close < lower or close > upper:
                self.position.close()
        else:
            if eom > 0:
                self.buy(size=1)

bt = Backtest(train_day, EoM_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True)
stats = bt.optimize(window=range(10, 30, 2), maximize='Equity Final [$]')
print(stats)


Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                    97.80658
Equity Final [$]                     3326.271
Equity Peak [$]                      3605.571
Return [%]                            10.8757
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                      2.6546
Volatility (Ann.) [%]                8.416087
Sharpe Ratio                          0.31542
Sortino Ratio                        0.432428
Calmar Ratio                         0.194587
Max. Drawdown [%]                  -13.642196
Avg. Drawdown [%]                   -1.631529
Max. Drawdown Duration      795 days 00:00:00
Avg. Drawdown Duration       54 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      33.900786
Worst Trade [%]                     33.900786
Avg. Trade [%]                    

##### Force Index (FI)


In [ ]:
class FI_Strategy(Strategy):
    window = 13

    def init(self):
        self.fi = self.I(calc_force_index, self.data, self.window)

    def next(self):
        fi = self.fi[-1]
        close = self.data.Close[-1]
        upper = close * 1.05
        lower = close * 0.95

        if self.position:
            if close < lower or close > upper:
                self.position.close()
        else:
            if fi > 0:
                self.buy(size=1)

bt = Backtest(train_day, FI_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True)
stats = bt.optimize(window=range(10, 30, 2), maximize='Equity Final [$]')
print(stats)


Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   97.507478
Equity Final [$]                     3331.018
Equity Peak [$]                      3610.318
Return [%]                          11.033933
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                    2.691759
Volatility (Ann.) [%]                8.400991
Sharpe Ratio                          0.32041
Sortino Ratio                        0.439393
Calmar Ratio                          0.19762
Max. Drawdown [%]                  -13.620853
Avg. Drawdown [%]                    -1.67845
Max. Drawdown Duration      795 days 00:00:00
Avg. Drawdown Duration       56 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      34.564501
Worst Trade [%]                     34.564501
Avg. Trade [%]                    

##### Money Flow Index (MFI)


In [ ]:
class MFI_Strategy(Strategy):
    window = 14

    def init(self):
        self.mfi = self.I(calc_money_flow_index, self.data, self.window)

    def next(self):
        mfi = self.mfi[-1]
        close = self.data.Close[-1]
        upper = close * 1.05
        lower = close * 0.95

        if self.position:
            if close < lower or close > upper or mfi > 80:
                self.position.close()
        else:
            if mfi < 20:
                self.buy(size=1)

bt = Backtest(train_day, MFI_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True)
stats = bt.optimize(window=range(10, 30, 2), maximize='Equity Final [$]')
print(stats)


Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                    32.20339
Equity Final [$]                      3028.44
Equity Peak [$]                      3218.279
Return [%]                              0.948
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                    0.239734
Volatility (Ann.) [%]                4.364485
Sharpe Ratio                         0.054928
Sortino Ratio                        0.073274
Calmar Ratio                          0.02967
Max. Drawdown [%]                   -8.080064
Avg. Drawdown [%]                   -1.085124
Max. Drawdown Duration      662 days 00:00:00
Avg. Drawdown Duration       73 days 00:00:00
# Trades                                    8
Win Rate [%]                             62.5
Best Trade [%]                      10.939948
Worst Trade [%]                     -12.85291
Avg. Trade [%]                    

##### Negative Volume Index (NVI)


In [ ]:
class NVI_Strategy(Strategy):

    def init(self):
        self.nvi = self.I(calc_negative_volume_index, self.data)

    def next(self):
        nvi = self.nvi[-1]
        close = self.data.Close[-1]
        upper = close * 1.05
        lower = close * 0.95

        if self.position:
            if close < lower or close > upper:
                self.position.close()
        else:
            if nvi > 0:
                self.buy(size=1)

bt = Backtest(train_day, NVI_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True)
stats = bt.run()
print(stats)


Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   99.800598
Equity Final [$]                      3321.12
Equity Peak [$]                       3600.42
Return [%]                             10.704
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                    2.614234
Volatility (Ann.) [%]                8.452775
Sharpe Ratio                         0.309275
Sortino Ratio                         0.42391
Calmar Ratio                         0.191303
Max. Drawdown [%]                   -13.66543
Avg. Drawdown [%]                    -1.63086
Max. Drawdown Duration      795 days 00:00:00
Avg. Drawdown Duration       54 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      33.187953
Worst Trade [%]                     33.187953
Avg. Trade [%]                    

##### On-Balance Volume (OBV)


In [ ]:
class OBV_Strategy(Strategy):

    def init(self):
        self.obv = self.I(calc_on_balance_volume, self.data)

    def next(self):
        obv = self.obv[-1]
        close = self.data.Close[-1]
        upper = close * 1.05
        lower = close * 0.95

        if self.position:
            if close < lower or close > upper:
                self.position.close()
        else:
            if obv > 0:
                self.buy(size=1)

bt = Backtest(train_day, OBV_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True)
stats = bt.run()
print(stats)


Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   99.800598
Equity Final [$]                      3321.12
Equity Peak [$]                       3600.42
Return [%]                             10.704
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                    2.614234
Volatility (Ann.) [%]                8.452775
Sharpe Ratio                         0.309275
Sortino Ratio                         0.42391
Calmar Ratio                         0.191303
Max. Drawdown [%]                   -13.66543
Avg. Drawdown [%]                    -1.63086
Max. Drawdown Duration      795 days 00:00:00
Avg. Drawdown Duration       54 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      33.187953
Worst Trade [%]                     33.187953
Avg. Trade [%]                    

##### Volume Price Trend (VPT)

In [ ]:
class VPT_Strategy(Strategy):

    def init(self):
        self.vpt = self.I(calc_volume_price_trend, self.data)

    def next(self):
        vpt = self.vpt[-1]
        close = self.data.Close[-1]
        upper = close * 1.05
        lower = close * 0.95

        if self.position:
            if close < lower or close > upper:
                self.position.close()
        else:
            if vpt > 0:
                self.buy(size=1)

bt = Backtest(train_day, VPT_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True)
stats = bt.run()
print(stats)


Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   99.800598
Equity Final [$]                      3321.12
Equity Peak [$]                       3600.42
Return [%]                             10.704
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                    2.614234
Volatility (Ann.) [%]                8.452775
Sharpe Ratio                         0.309275
Sortino Ratio                         0.42391
Calmar Ratio                         0.191303
Max. Drawdown [%]                   -13.66543
Avg. Drawdown [%]                    -1.63086
Max. Drawdown Duration      795 days 00:00:00
Avg. Drawdown Duration       54 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      33.187953
Worst Trade [%]                     33.187953
Avg. Trade [%]                    

##### Volume Weighted Average Price (VWAP)


In [ ]:
class VWAP_Strategy(Strategy):
    window = 14

    def init(self):
        self.vwap = self.I(calc_volume_weighted_average_price, self.data, self.window)

    def next(self):
        close = self.data.Close[-1]
        vwap = self.vwap[-1]
        upper = close * 1.05
        lower = close * 0.95

        if self.position:
            if close < lower or close > upper:
                self.position.close()
        else:
            if close > vwap:
                self.buy(size=1)

bt = Backtest(data_day, VWAP_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True)
stats = bt.optimize(window=range(5, 30, 2), maximize='Equity Final [$]')
print(stats)


Start                     2018-08-13 07:00:00
End                       2024-08-30 09:00:00
Duration                   2209 days 02:00:00
Exposure Time [%]                   98.603723
Equity Final [$]                      3363.85
Equity Peak [$]                       3603.45
Return [%]                          12.128333
Buy & Hold Return [%]               39.666737
Return (Ann.) [%]                    1.952285
Volatility (Ann.) [%]                 8.27641
Sharpe Ratio                         0.235886
Sortino Ratio                        0.321787
Calmar Ratio                         0.104531
Max. Drawdown [%]                  -18.676546
Avg. Drawdown [%]                   -1.962227
Max. Drawdown Duration     1152 days 00:00:00
Avg. Drawdown Duration       83 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      37.722254
Worst Trade [%]                     37.722254
Avg. Trade [%]                    